In [1]:
%load_ext jupyter_black

In [2]:
import os
from shapely.geometry import Polygon

os.environ["USE_PYGEOS"] = "0"
import geopandas
import shutil
import numpy as np
import napari
import os
import spatialdata as sd
from spatialdata.transformations import (
    Affine,
    Sequence,
    Identity,
    align_elements_using_landmarks,
    get_transformation,
    set_transformation,
)
from napari_spatialdata import Interactive

In [3]:
print("current working directory:", os.getcwd())
SPATIALDATA_SANDBOX_PATH = "spatialdata-sandbox"
assert os.path.isdir(
    SPATIALDATA_SANDBOX_PATH
), f"{SPATIALDATA_SANDBOX_PATH} not found, please use symlinks to make it available"
GENERATED_DATA_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "generated_data/xenium_visium_integration"
)
assert os.path.isdir(
    GENERATED_DATA_PATH
), f"{GENERATED_DATA_PATH} not found, please use symlinks to make it available"


XE_REP1_PATH = os.path.join(SPATIALDATA_SANDBOX_PATH, "xenium_rep1_io/data.zarr")
XE_REP2_PATH = os.path.join(SPATIALDATA_SANDBOX_PATH, "xenium_rep2_io/data.zarr")
VISIUM_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "visium_associated_xenium_io/data.zarr"
)

XE_REP1_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "xe_rep1_roi.zarr")
XE_REP2_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "xe_rep2_roi.zarr")
VISIUM_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "visium_roi.zarr")

LANDMARKS_SDATA_PATH = os.path.join(GENERATED_DATA_PATH, "sandbox.zarr")

current working directory: /Users/macbook/embl/projects/basel/spatialdata-sandbox/notebooks


In [4]:
# we delete some elements (in-memory only) because we will not need them in napari
xe_rep1_sdata = sd.read_zarr(XE_REP1_PATH)
del xe_rep1_sdata.images["morphology_focus"]
del xe_rep1_sdata.shapes["cell_boundaries"]
del xe_rep1_sdata.shapes["nucleus_boundaries"]
xe_rep2_sdata = sd.read_zarr(XE_REP2_PATH)
del xe_rep2_sdata.images["morphology_focus"]
del xe_rep2_sdata.shapes["cell_boundaries"]
del xe_rep2_sdata.shapes["nucleus_boundaries"]
visium_sdata = sd.read_zarr(VISIUM_PATH)

xe_rep1_roi_sdata = sd.read_zarr(XE_REP1_ROI_PATH)
del xe_rep1_roi_sdata.shapes["cell_boundaries"]
del xe_rep1_roi_sdata.shapes["nucleus_boundaries"]
xe_rep2_roi_sdata = sd.read_zarr(XE_REP2_ROI_PATH)
del xe_rep2_roi_sdata.shapes["cell_boundaries"]
del xe_rep2_roi_sdata.shapes["nucleus_boundaries"]
visium_roi_sdata = sd.read_zarr(VISIUM_ROI_PATH)

landmarks_sdata = sd.read_zarr(LANDMARKS_SDATA_PATH)

INFO     Instance key `cell_id` could be of type `pd.Categorical`. Consider casting it.                            
INFO     Instance key `cell_id` could be of type `pd.Categorical`. Consider casting it.                            


In [8]:
# remove the cells, for xenium rep 2, inside the box (so already present in xenium rep 1)
xe_rep2_sdata.shapes["cell_circles"] = xe_rep2_sdata["cell_circles"][
    ~xe_rep2_sdata["cell_circles"].index.isin(xe_rep2_roi_sdata["cell_circles"].index)
]

In [7]:
Interactive([xe_rep1_sdata, xe_rep2_sdata, visium_sdata, landmarks_sdata], points=False)

/Users/macbook/miniconda3/envs/ome/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
2023-03-29 01:38:11.780 | WARNING  | napari_spatialdata.interactive:_add_circles:258 - Too many shapes 167780 > 10000, using points instead of ellipses. Size will stop being correct beyond a certain zoom level


ValueError: Partial annotation is support only when the annotation table contains instances not in circles.